# huggingface/trl SFTTrainerを使用したファインチューニング
参考：https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?huggingface%2Ftrl%20SFTTrainer  
※ ディスク領域が必要なので、このNBを実行する場合は、gitクローンする段階で/mnt/tmp（※ 一時領域）などに配置すると良い。

## 準備

### 必要なパッケージをインストール

#### サンドボックス

In [ ]:
!pip cache purge
# !pip install --upgrade XXXX

#### 実際にインストールしたもの一覧

- datasets : 自然言語処理のサンプルデータセットを簡単に取り扱えるライブラリ
- transformers : Hugging Face の transformers ライブラリ
- accelerate : Hugging Face の accelerate ライブラリは、TPU/GPU/CPUでの実行を同じコードで記述できる
- trl : Hugging Face の trl (Transformer Reinforcement Learning) ライブラリ（SFT用）
- peft : Hugging Face の peft (Parameter-Efficient Fine Tuning) ライブラリ（LoRA用）
```bash
!pip install --upgrade datasets
!pip install --upgrade transformers
!pip install --upgrade accelerate
!pip install --upgrade trl
!pip install --upgrade peft
```

#### 以下のエラーが出るので、インストール後に一旦再起動
```
ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`
```
参考：https://github.com/huggingface/transformers/issues/24147

#### インストール結果の確認

In [ ]:
!pip list

### インストールしたパッケージをインポート

In [1]:
import torch

import datasets
from datasets import load_dataset

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

import trl
from trl import DataCollatorForCompletionOnlyLM, SFTTrainer

import peft
from peft import LoraConfig

In [2]:
print(peft.__version__, transformers.__version__, trl.__version__)

0.16.0 4.53.2 0.19.1


### 使用する変数を定義する

In [3]:
# DATA
TEST_FILE_PATH = 'finetuning/data/test.jsonl'

# MODEL
MODEL_URI                        = 'meta-llama/Llama-3.2-1B-Instruct'
MODEL_PATH      =           'finetuning/models/Llama-3.2-1B-Instruct'
FTED_MODEL_PATH = 'finetuning/finetuned_models/Llama-3.2-1B-Instruct'

### モデル・トークナイザ

#### NotebookからHugging Face Model Hubとやり取り

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

### モデルとトークナイザのダウンロードとセーブ
初回のみなのでMarkdownにしておく。

#### モデルとトークナイザのダウンロードと

```python
# model
model = transformers.AutoModelForCausalLM.from_pretrained(
    MODEL_URI,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    force_download=True
  )
model.save_pretrained(MODEL_PATH) 

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_URI)
tokenizer.save_pretrained(MODEL_PATH) 
```

#### モデルとトークナイザのセーブ

```python
# model
model = transformers.AutoModelForCausalLM.from_pretrained(
    MODEL_URI,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    force_download=True
  )
model.save_pretrained(MODEL_PATH) 

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_URI)
tokenizer.save_pretrained(MODEL_PATH)
```

## ファインチューニング

### データの読み込み

#### 間違った方法
- なお、以下で表示される結果の「Datasetのtrain」と「DatasetDictのtrain」は対応している訳ではない。
- これは、`datasets.DatasetDict({'train': dataset})`のtrain → xxxxx に変えて実行すると解る。

In [4]:
dataset = datasets.load_dataset("json", data_files=TEST_FILE_PATH)
dataset = datasets.DatasetDict({'train': dataset})
dataset

DatasetDict({
    train: DatasetDict({
        train: Dataset({
            features: ['answer', 'question', 'context'],
            num_rows: 11
        })
    })
})

#### 正しい方法

In [5]:
dataset = datasets.load_dataset("json", data_files=TEST_FILE_PATH, split="train")
dataset = datasets.DatasetDict({'train': dataset})
dataset

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 11
    })
})

### SFTTrainerの定義

#### model, tokenizer
モデルとトークナイザのロード

In [6]:
# model
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [7]:
tokenizer.eos_token

'<|eot_id|>'

#### training_args

In [8]:
#training_args = TrainingArguments('test_trainer') # output_dirの指定になる。

# 「OutOfMemoryError: CUDA out of memory.」への対応
training_args = TrainingArguments(
    output_dir='test_trainer',
    per_device_train_batch_size=2, # 8, 4, 2
    num_train_epochs=1000,         # ← ここを増やす
    gradient_checkpointing=True,
    bf16=True, #fp16=True          # NVIDIA Ampere 以降の GPU なら bf16=True も検討
)

#### formatting_func

In [9]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['question'])):
        text = f"Please answer the question.\n\n### question\n{example['question'][i]}\n\n### answer\n{example['answer'][i]}<|eot_id|>"
        output_texts.append(text)
    return output_texts

#### data_collator

In [10]:
response_template = '### answer\n'
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

#### peft_config
LoRAでないとLlama-3.2-1B-Instruct@A10はメモリ不足でトレーナーを動作させることができなかった（環境の問題？）。

In [11]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

#### SFTTrainerの定義

In [12]:
trainer = SFTTrainer(
    model,
    args=training_args,
    train_dataset=dataset['train'], 
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    peft_config=peft_config,
)

Applying formatting function to train dataset:   0%|          | 0/11 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


#### SFTTrainer.train_datasetを確認

In [13]:
print(trainer.train_dataset)
# SFTTrainerの出力が途中で止まることがあるので4件目を確認
print(tokenizer.decode(trainer.train_dataset[3]['input_ids']))

Dataset({
    features: ['answer', 'question', 'context', 'text', 'input_ids'],
    num_rows: 11
})
<|begin_of_text|>Please answer the question.

### question
ゼノリア星の気候はどうですか？

### answer
ゼノリア星は温暖な地域と氷結地帯が混在し、季節によって光の強度が大きく変化します。<|eot_id|>


### SFTTrainer.trainで実行

#### OutOfMemoryError: CUDA out of memory. 対策
要るか要らないか解らないので、取り敢えずMarkdownで。

##### 設定

```python
#`use_cache=True`は勾配チェックポイントと互換性が無いので併用できない。
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
```

##### 確認1

```python
torch.cuda.memory_summary()
```

##### 確認2

```python
torch.cuda.memory_allocated()
```

#### 実際にSFTを実行

In [14]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [15]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
500,0.350000
1000,0.001000
1500,0.000500
2000,0.000300
2500,0.000200
3000,0.000100
3500,0.000100
4000,0.000100
4500,0.000100
5000,0.000100


TrainOutput(global_step=6000, training_loss=0.02937101301861306, metrics={'train_runtime': 524.8493, 'train_samples_per_second': 20.958, 'train_steps_per_second': 11.432, 'total_flos': 3828437761167360.0, 'train_loss': 0.02937101301861306})

#### SFTされたモデルとトークナイザのセーブ

In [16]:
tokenizer.save_pretrained(FTED_MODEL_PATH) # ,device_map="auto", torch_dtype=torch.float16)
trainer.save_model(FTED_MODEL_PATH)

## SFTされたモデルで推論を実行

### SFTされたモデルとトークナイザのロード

In [ ]:
tokenizer.from_pretrained(FTED_MODEL_PATH)
trainer.from_pretrained(FTED_MODEL_PATH)

### プロンプトをトークン化
`pt`は、PyTorch テンソル（torch.Tensor）の意味

In [17]:
inputs = tokenizer('ゼノリア星に住む主な種族は何ですか？', return_tensors='pt').to(model.device)
print(inputs) # テンソルのディクショナリになっている。

{'input_ids': tensor([[128000, 108270, 101335, 104612,  78519,  20230, 101987, 104004,  36668,
          26854, 104091,  71869,  15682,  99849, 112130,  11571]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


### 推論を実行
- 推論時なので勾配計算しない`no_grad`プロックに入れて実行
- `**inputs`として渡すと「ディクショナリのキー・値」が「キーワード引数・引数」に展開される。

In [18]:
model.gradient_checkpointing_disable()  # checkpointing をオフにする

with torch.no_grad():
    tokens = model.generate(
        **inputs,
        max_new_tokens=64,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.05,
        pad_token_id=tokenizer.pad_token_id,
    )

### 推論結果を表示
推論結果をデトークン化して表示。

In [19]:
output = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(output)

ゼノリア星に住む主な種族は何ですか？
ゼノリア星に住む主な種族はゼノス族です。ゼノス族はゼノリア星の最も重要なものです。ゼノス族はゼノリア星の最も重要なものです。ゼノス族はゼノリア星の最
